In [3]:
import pandas as pd
import numpy as np
from pydataset import data

In [20]:
import pyspark
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, concat_ws, lit
from pyspark.sql.functions import regexp_extract, regexp_replace
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# 1
Create a spark data frame that contains your favorite programming languages.

    The name of the column should be language
    View the schema of the dataframe
    Output the shape of the dataframe
    Show the first 5 records in the dataframe

In [ ]:
#mas tarde

# 2
Load the mpg dataset as a spark dataframe.

    Create 1 column of output that contains a message like the one below:


    The 1999 audi a4 has a 4 cylinder engine.
    For each vehicle.

   

In [4]:
mpg = spark.createDataFrame(data("mpg"))

In [23]:
mpg.select(concat
           (lit("The "), 
            mpg.year, 
            lit(" "), 
            mpg.manufacturer, 
            lit(" "), 
            mpg.model, 
            lit(" has a "), 
            mpg.cyl, 
            lit(" engine.")).alias("The Vehicle's Description")).show(5,truncate=False)

+--------------------------------+
|The Vehicle's Description       |
+--------------------------------+
|The 1999 audi a4 has a 4 engine.|
|The 1999 audi a4 has a 4 engine.|
|The 2008 audi a4 has a 4 engine.|
|The 2008 audi a4 has a 4 engine.|
|The 1999 audi a4 has a 6 engine.|
+--------------------------------+
only showing top 5 rows



    Transform the trans column so that it only contains either manual or auto.

In [28]:
mpg.select(
    regexp_extract("trans", r"(auto|manual)", 1).alias("trans_type")).show(truncate=False)


+----------+
|trans_type|
+----------+
|auto      |
|manual    |
|manual    |
|auto      |
|auto      |
|manual    |
|auto      |
|manual    |
|auto      |
|manual    |
|auto      |
|auto      |
|manual    |
|auto      |
|manual    |
|auto      |
|auto      |
|auto      |
|auto      |
|auto      |
+----------+
only showing top 20 rows



In [19]:
mpg.select(mpg.trans).show(5)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
+----------+
only showing top 5 rows



# 3
Load the tips dataset as a spark dataframe.

    What percentage of observations are smokers?

In [29]:
tips = spark.createDataFrame(data("tips"))

In [32]:
tips.where(tips["smoker"] == "Yes").show(5)

+----------+----+----+------+---+------+----+
|total_bill| tip| sex|smoker|day|  time|size|
+----------+----+----+------+---+------+----+
|     38.01| 3.0|Male|   Yes|Sat|Dinner|   4|
|     11.24|1.76|Male|   Yes|Sat|Dinner|   2|
|     20.29|3.21|Male|   Yes|Sat|Dinner|   2|
|     13.81| 2.0|Male|   Yes|Sat|Dinner|   2|
|     11.02|1.98|Male|   Yes|Sat|Dinner|   2|
+----------+----+----+------+---+------+----+
only showing top 5 rows



In [37]:
tips.describe().show()

+-------+------------------+-----------------+------+------+----+------+------------------+
|summary|        total_bill|              tip|   sex|smoker| day|  time|              size|
+-------+------------------+-----------------+------+------+----+------+------------------+
|  count|               244|              244|   244|   244| 244|   244|               244|
|   mean|19.785942622950813| 2.99827868852459|  null|  null|null|  null| 2.569672131147541|
| stddev| 8.902411954856856|1.383638189001182|  null|  null|null|  null|0.9510998047322344|
|    min|              3.07|              1.0|Female|    No| Fri|Dinner|                 1|
|    max|             50.81|             10.0|  Male|   Yes|Thur| Lunch|                 6|
+-------+------------------+-----------------+------+------+----+------+------------------+



In [42]:
tips.groupBy(tips.smoker).agg((count(tips.smoker)/244).alias("Percentage of 244 Customers")).show()

+------+---------------------------+
|smoker|Percentage of 244 Customers|
+------+---------------------------+
|    No|         0.6188524590163934|
|   Yes|        0.38114754098360654|
+------+---------------------------+



    Create a column that contains the tip percentage

In [44]:
tips.select((tips.tip/tips.total_bill).alias("Tip as Percentge of Bill")).show(5)

+------------------------+
|Tip as Percentge of Bill|
+------------------------+
|     0.05944673337257211|
|     0.16054158607350097|
|     0.16658733936220846|
|      0.1397804054054054|
|     0.14680764538430255|
+------------------------+
only showing top 5 rows



In [45]:
tip_percent = tips.select((tips.tip/tips.total_bill).alias("Tip as Percentge of Bill"))

    Calculate the average tip percentage for each combination of sex and smoker.

In [49]:
sexy_smokers = tips.groupby("sex", "smoker")

In [55]:
sexy_smokers.agg(avg('tip')/avg('total_bill')).show()

+------+------+----------------------------+
|   sex|smoker|(avg(tip) / avg(total_bill))|
+------+------+----------------------------+
|  Male|    No|          0.1573121500195338|
|  Male|   Yes|         0.13691878510474398|
|Female|    No|         0.15318918255461905|
|Female|   Yes|         0.16306234935189712|
+------+------+----------------------------+



# 4
Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [57]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)

    Convert the temperatures to farenheight.

In [59]:
weather.describe().show()

+-------+----------+-----------------+-----------------+-----------------+------------------+-------+
|summary|      date|    precipitation|         temp_max|         temp_min|              wind|weather|
+-------+----------+-----------------+-----------------+-----------------+------------------+-------+
|  count|      1461|             1461|             1461|             1461|              1461|   1461|
|   mean|      null|3.029431895961668|16.43908281998631| 8.23477070499658|3.2411362080766604|   null|
| stddev|      null|6.680194322314739|7.349758097360176|5.023004179961265|1.4378250588746198|   null|
|    min|2012-01-01|              0.0|             -1.6|             -7.1|               0.4|drizzle|
|    max|2015-12-31|             55.9|             35.6|             18.3|               9.5|    sun|
+-------+----------+-----------------+-----------------+-----------------+------------------+-------+



In [61]:
weather.select((weather.temp_max*9/5+32).alias(.show(5)

+---------------------------+
|(((temp_max * 9) / 5) + 32)|
+---------------------------+
|                      55.04|
|                      51.08|
|                      53.06|
|                      53.96|
|                      48.02|
+---------------------------+
only showing top 5 rows



    Which month has the most rain, on average?

    Which year was the windiest?

    What is the most frequent type of weather in January?

    What is the average high and low tempurature on sunny days in July in 2013 and 2014?

    What percentage of days were rainy in q3 of 2015?

    For each year, find what percentage of days it rained (had non-zero precipitation).